In [ ]:
#@title 🔧 Quick NumPy Check (Optional)
# Run this cell first if you want to check for NumPy compatibility issues
# This is optional - you can skip to Cell 1 if you prefer

try:
    import numpy as np
    numpy_version = np.__version__
    numpy_major = int(numpy_version.split('.')[0])
    
    print(f"Current NumPy version: {numpy_version}")
    
    if numpy_major >= 2:
        print("⚠️ WARNING: NumPy 2.x detected!")
        print("This may cause compatibility issues with transformers.")
        print("If you get errors in Cell 3, restart runtime and try again.")
    else:
        print("✅ NumPy version looks compatible")
        
except ImportError:
    print("NumPy not installed yet - this is normal")
    
print("\n🚀 Ready to proceed! Continue with Cell 1 below.")
print("(Remember: if you get errors, just restart runtime and try again)")

# Also check if we're in Colab
if 'google.colab' in str(type(get_ipython())):
    print("\n✅ Running in Google Colab")
else:
    print("\n⚠️ Not running in Google Colab - some features may not work")


# Zonos Model: Google Colab Notebook

This notebook will help you set up and run the Zonos model in Google Colab. The model will be automatically downloaded from [Hugging Face](https://huggingface.co/Wamp1re-Ai/Zonos-v0.1-transformer).

## Steps:
1. Clone or upload the Zonos repository and download the model from HuggingFace.
2. Install dependencies.
3. Run the model with example code.
4. (Optional) Upload your own audio files for inference.
5. Use Cloudflare subdomain for sharing your Gradio interface.

## ⚠️ Important: NumPy Compatibility Issue

**If you get a NumPy/Transformers error in Cell 3**, this is a known compatibility issue:
- **Problem**: Google Colab sometimes loads NumPy 2.x which is incompatible with transformers
- **Symptom**: Error mentioning `_center` or NumPy in transformers import
- **Solution**: **Restart runtime** (Runtime → Restart runtime) and re-run cells 1-3

This is **normal** and **easy to fix** - just restart the runtime when prompted!

In [ ]:
#@title 1. Clone the repository and download the model
import os

# Clone the Zonos repository (use the correct URL)
if not os.path.exists('Zonos'):
    !git clone https://github.com/Wamp1re-Ai/Zonos.git  # Change this to your actual GitHub username
    print("Repository cloned successfully!")
else:
    print("Repository already exists!")

%cd Zonos

# Install system dependencies first (eSpeak is required for phonemization)
!apt-get update -qq
!apt-get install -y espeak-ng git-lfs

# Initialize git LFS
!git lfs install

print("System dependencies installed successfully!")

In [ ]:
#@title 2. Install dependencies
# Install required packages efficiently and avoid dependency conflicts
import subprocess
import sys
import os

def install_package(package, use_uv=False):
    """Install a package with better error handling"""
    try:
        if use_uv:
            subprocess.check_call(["uv", "pip", "install", package, "--quiet"], env={**os.environ, "UV_SYSTEM_PYTHON": "1"})
        else:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])
        return True
    except subprocess.CalledProcessError as e:
        print(f"Failed to install {package}: {e}")
        return False

# Install UV for faster package management
print("Installing UV for faster package management...")
try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "uv", "--quiet"])
    print("✓ UV installed successfully!")
    use_uv = True
except subprocess.CalledProcessError:
    print("Failed to install UV, falling back to pip")
    use_uv = False

# Don't upgrade pip/setuptools in Colab - it causes conflicts
print("Skipping pip upgrade to avoid dependency conflicts in Colab...")

# Check if we're in Colab and use pre-installed torch if available
IN_COLAB = 'google.colab' in sys.modules
print(f"Running in Google Colab: {IN_COLAB}")

if IN_COLAB:
    # Use Colab's pre-installed PyTorch to save time
    print("Using Colab's pre-installed PyTorch...")
    try:
        import torch
        import torchaudio
        print(f"✓ PyTorch {torch.__version__} already available")
        print(f"✓ TorchAudio {torchaudio.__version__} already available")
        torch_installed = True
    except ImportError:
        print("PyTorch not found, will install...")
        torch_installed = False
else:
    torch_installed = False

# Check current numpy version BEFORE any installations
print("\n🔍 Checking current NumPy version...")
try:
    import numpy as np
    current_numpy = np.__version__
    numpy_major, numpy_minor = map(int, current_numpy.split('.')[:2])
    print(f"Current NumPy: {current_numpy}")
    
    if numpy_major >= 2:
        print("⚠️ NumPy 2.x detected - this will cause transformers compatibility issues!")
        numpy_needs_fix = True
    else:
        print("✓ NumPy version is compatible")
        numpy_needs_fix = False
except ImportError:
    print("NumPy not installed yet")
    numpy_needs_fix = False

# Force install compatible numpy version if needed
if numpy_needs_fix or not IN_COLAB:
    print("\n🔧 Installing compatible NumPy version...")
    if use_uv:
        subprocess.check_call(["uv", "pip", "install", "numpy==1.26.4", "--force-reinstall", "--quiet"], env={**os.environ, "UV_SYSTEM_PYTHON": "1"})
    else:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy==1.26.4", "--force-reinstall", "--quiet"])
    print("✓ NumPy 1.26.4 installed")
    
    if numpy_needs_fix:
        print("\n⚠️ IMPORTANT: NumPy was downgraded from 2.x to 1.26.4")
        print("This may not take effect until you restart the runtime.")
        print("If you get import errors in the next cell, please:")
        print("  1. Runtime → Restart runtime")
        print("  2. Re-run cells 1, 2, and 3")

# Install core dependencies with compatible versions
packages = [
    "transformers>=4.45.0,<4.50.0",  # Pin to avoid compatibility issues
    "gradio>=4.0.0,<5.0.0", 
    "huggingface-hub>=0.20.0",
    "soundfile>=0.12.1",
    "phonemizer>=3.2.0",
    "inflect>=7.0.0",
    "scipy"
]

# Add torch packages if not already installed
if not torch_installed:
    packages = ["torch>=2.0.0", "torchaudio>=2.0.0"] + packages

print(f"\nInstalling {len(packages)} core dependencies...")
failed_packages = []

for package in packages:
    print(f"Installing {package}...")
    if not install_package(package, use_uv):
        failed_packages.append(package)

if failed_packages:
    print(f"\n⚠️ Failed to install: {failed_packages}")
    print("Continuing anyway - some packages might work...")

# Install optional dependencies with better error handling
print("\nInstalling optional dependencies (these may fail - it's normal)...")
optional_packages = [
    ("flash-attn>=2.0.0", "Flash Attention for faster inference"),
    ("mamba-ssm>=1.0.0", "Mamba State Space Models"), 
    ("causal-conv1d>=1.0.0", "Causal Conv1D for Mamba")
]

optional_success = 0
for package, description in optional_packages:
    print(f"Attempting to install {package} ({description})...")
    if install_package(package, use_uv):
        print(f"  ✓ {package} installed successfully")
        optional_success += 1
    else:
        print(f"  ✗ {package} installation failed (optional - continuing)")

# Install the project itself
print("\nInstalling Zonos package...")
try:
    if use_uv:
        subprocess.check_call(["uv", "pip", "install", "-e", ".", "--quiet"], env={**os.environ, "UV_SYSTEM_PYTHON": "1"})
    else:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-e", ".", "--quiet"])
    print("✓ Zonos package installed successfully!")
except subprocess.CalledProcessError as e:
    print(f"❌ Failed to install Zonos package: {e}")
    print("Trying alternative installation...")
    # Alternative: add current directory to Python path
    current_dir = os.getcwd()
    if current_dir not in sys.path:
        sys.path.insert(0, current_dir)
    print(f"Added {current_dir} to Python path")

# Verify critical imports work
print("\n🔍 Verifying critical imports...")
try:
    import numpy as np
    print(f"✓ NumPy {np.__version__} working")
except Exception as e:
    print(f"❌ NumPy import failed: {e}")

try:
    import transformers
    print(f"✓ Transformers {transformers.__version__} working")
except Exception as e:
    print(f"❌ Transformers import failed: {e}")
    if "_center" in str(e) or "numpy" in str(e).lower():
        print("\n🔧 This looks like the NumPy 2.x compatibility issue!")
        print("Please restart the runtime and try again.")

print("\n✅ Dependency installation complete!")
print("\n📝 Installation Summary:")
print(f"  - Package manager: {'UV' if use_uv else 'pip'}")
print(f"  - Core packages: {len(packages) - len(failed_packages)}/{len(packages)} successful")
print(f"  - Optional packages: {optional_success}/{len(optional_packages)} successful")
if failed_packages:
    print(f"  - Failed packages: {failed_packages}")
print(f"  - Zonos package: Installed")
print(f"  - NumPy compatibility: {'Fixed' if numpy_needs_fix else 'OK'}")

In [ ]:
#@title 3. Load and run the Zonos model
import sys
import os
import subprocess

# Make sure we can import zonos modules
if '/content/Zonos' not in sys.path:
    sys.path.insert(0, '/content/Zonos')

# CRITICAL: Check numpy compatibility first - this must be done before importing transformers
print("🔧 Checking NumPy compatibility...")
try:
    import numpy as np
    numpy_version = np.__version__
    numpy_major, numpy_minor = map(int, numpy_version.split('.')[:2])
    print(f"Current NumPy: {numpy_version}")
    
    # Check if numpy version is compatible - NumPy 2.x causes issues
    if numpy_major >= 2:
        print("\n⚠️ NumPy 2.x detected - attempting automatic fix...")
        print("Installing compatible NumPy version...")
        
        try:
            # Force reinstall compatible numpy
            subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy==1.26.4", "--force-reinstall", "--quiet"])
            print("✓ NumPy 1.26.4 installed successfully")
            
            # Clear the module cache and reimport
            if 'numpy' in sys.modules:
                del sys.modules['numpy']
            if 'np' in globals():
                del globals()['np']
                
            # Re-import numpy
            import numpy as np
            numpy_version = np.__version__
            numpy_major = int(numpy_version.split('.')[0])
            print(f"Updated NumPy: {numpy_version}")
            
            if numpy_major >= 2:
                print("\n❌ CRITICAL ERROR: NumPy 2.x still detected after reinstall!")
                print("NumPy 2.x is incompatible with transformers library.")
                print("\n🔄 MANUAL SOLUTION REQUIRED:")
                print("1. Click 'Runtime' in the top menu")
                print("2. Click 'Restart runtime'")
                print("3. Run Cell 1 (Clone repository)")
                print("4. Run Cell 2 (Install dependencies)")
                print("5. Run Cell 3 (this cell) again")
                print("\n💡 Why this happens:")
                print("- Colab sometimes has NumPy 2.x pre-loaded in memory")
                print("- Restarting clears memory and uses our installed NumPy 1.26.4")
                print("- This is a known compatibility issue with transformers")
                raise RuntimeError("NumPy 2.x compatibility issue - runtime restart required")
            else:
                print(f"✓ NumPy {numpy_version} is now compatible!")
                
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed to automatically fix NumPy: {e}")
            print("\n🔄 MANUAL SOLUTION REQUIRED:")
            print("1. Click 'Runtime' in the top menu")
            print("2. Click 'Restart runtime'")
            print("3. Run Cell 1 (Clone repository)")
            print("4. Run Cell 2 (Install dependencies)")
            print("5. Run Cell 3 (this cell) again")
            raise RuntimeError("NumPy 2.x compatibility issue - runtime restart required")
    else:
        print(f"✓ NumPy {numpy_version} is compatible")
except ImportError:
    print("❌ NumPy not found! Please run Cell 2 first.")
    raise
except Exception as e:
    if "restart" in str(e) or "compatibility" in str(e):
        raise
    print(f"❌ NumPy error: {e}")
    raise

# Import PyTorch
print("\n📦 Loading PyTorch...")
try:
    import torch
    import torchaudio
    print(f"✓ PyTorch {torch.__version__} loaded successfully")
    print(f"✓ TorchAudio {torchaudio.__version__} loaded successfully")
except Exception as e:
    print(f"❌ PyTorch import error: {e}")
    raise

# Import transformers with enhanced error handling
print("\n🤗 Loading Transformers...")
try:
    import transformers
    print(f"✓ Transformers {transformers.__version__} loaded successfully")
except ImportError as e:
    error_msg = str(e)
    if "_center" in error_msg or "numpy" in error_msg.lower():
        print(f"❌ NumPy/Transformers compatibility error: {e}")
        print("\n🔧 DETECTED: NumPy 2.x compatibility issue")
        print("\n📋 REQUIRED SOLUTION:")
        print("┌─────────────────────────────────────┐")
        print("│ 1. Runtime → Restart runtime        │")
        print("│ 2. Run Cell 1 (Clone)              │")
        print("│ 3. Run Cell 2 (Dependencies)       │")
        print("│ 4. Run Cell 3 (Model) again        │")
        print("└─────────────────────────────────────┘")
        print("\n💡 This happens because:")
        print("- NumPy 2.x was loaded before we could replace it")
        print("- Restarting runtime will use our NumPy 1.26.4")
        print("- This is the only way to fix this compatibility issue")
        raise RuntimeError("NumPy/Transformers compatibility issue - restart required")
    else:
        print(f"❌ Transformers import error: {e}")
        print("\nTry:")
        print("1. Restart runtime")
        print("2. Re-run all cells from the beginning")
        raise

# Now try to import Zonos modules
print("\n🎵 Loading Zonos modules...")
try:
    from zonos.model import Zonos
    from zonos.conditioning import make_cond_dict, supported_language_codes
    from zonos.utils import DEFAULT_DEVICE
    print("✓ Zonos modules imported successfully!")
except ImportError as e:
    print(f"❌ Zonos import error: {e}")
    print("\nTroubleshooting:")
    print("- Make sure Cell 2 (dependency installation) completed successfully")
    print("- Check that you're in the Zonos directory")
    print("- Try restarting runtime and running from Cell 1")
    
    # Try importing individual modules to diagnose the issue
    print("\n🔍 Diagnostic imports:")
    try:
        from zonos import model
        print("✓ zonos.model imported")
    except Exception as e2:
        print(f"❌ zonos.model failed: {e2}")
    
    try:
        from zonos import conditioning
        print("✓ zonos.conditioning imported")
    except Exception as e2:
        print(f"❌ zonos.conditioning failed: {e2}")
    
    raise

# Set device (use GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n🖥️ Using device: {device}")

# Check GPU memory if using CUDA
if device.type == 'cuda':
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    # Clear GPU cache
    torch.cuda.empty_cache()
    print("✓ GPU cache cleared")

# Load the model from HuggingFace
model_name = "Zyphra/Zonos-v0.1-transformer"
print(f"\n📥 Loading model: {model_name}")
print("This may take 2-5 minutes for the first time...")

try:
    model = Zonos.from_pretrained(model_name, device=device)
    model.requires_grad_(False).eval()
    print("✅ Model loaded successfully!")
    
    # Show available conditioning options
    if hasattr(model, 'prefix_conditioner') and hasattr(model.prefix_conditioner, 'conditioners'):
        print("\n🎛️ Available conditioning options:")
        for c in model.prefix_conditioner.conditioners:
            print(f"  - {c.name}")
    
    # Show supported languages
    print(f"\n🌍 Supported languages: {supported_language_codes}")
    
    # Show model info
    total_params = sum(p.numel() for p in model.parameters())
    print(f"\n📊 Model info:")
    print(f"  - Total parameters: {total_params:,}")
    print(f"  - Device: {next(model.parameters()).device}")
    print(f"  - Dtype: {next(model.parameters()).dtype}")
            
    print("\n🎉 Setup complete! You can now use the model in the cells below.")
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("\n🔧 Troubleshooting tips:")
    print("1. Make sure you have a stable internet connection")
    print("2. Check if you have enough GPU/RAM memory")
    print("3. Try restarting the runtime and running from the beginning")
    print("4. Ensure you're using a GPU runtime for best performance")
    
    # Show memory usage for debugging
    if device.type == 'cuda':
        print(f"\n💾 GPU Memory Usage:")
        print(f"  - Allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
        print(f"  - Reserved: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")
    
    raise

In [ ]:
#@title 4. Generate Speech with Text-to-Speech (Uses Cloned Voice if Available)

# Import necessary modules for conditional generation
from google.colab import files
import IPython.display as ipd
import numpy as np

#@markdown ### Text and language settings
text = "Hello, this is Zonos text-to-speech model. How can I help you today?" #@param {type:"string"}
language = "en-us" #@param ["en-us", "en-gb", "fr-fr", "es-es", "de-de", "it-it", "ja-jp", "zh-cn"]

#@markdown ### Voice Settings
#@markdown The system will automatically use your cloned voice if you uploaded audio in the previous cell
use_cloned_voice = True #@param {type:"boolean"}

# Check if we have a cloned voice available
speaker_embedding = None
voice_info = ""

if use_cloned_voice and 'custom_speaker_embedding' in globals():
    speaker_embedding = custom_speaker_embedding
    if 'speaker_audio_info' in globals():
        info = speaker_audio_info
        voice_info = f"🎭 Using CLONED VOICE from '{info['filename']}' ({info['duration']:.1f}s)"
    else:
        voice_info = "🎭 Using CLONED VOICE from uploaded audio"
    print(voice_info)
else:
    if 'custom_speaker_embedding' in globals():
        print("🔘 Cloned voice available but disabled. Enable 'use_cloned_voice' to use it.")
    else:
        print("🎤 Using DEFAULT VOICE (no cloned voice uploaded)")
        print("💡 Tip: Run the 'Voice Cloning' cell above to upload your own voice!")
    voice_info = "🎤 Using default voice"

# Set a random seed for reproducibility
torch.manual_seed(42)

# Create conditioning dictionary
print("\n🔧 Creating conditioning...")
try:
    cond_dict = make_cond_dict(
        text=text,
        language=language,
        speaker=speaker_embedding,
        device=device,
        # Use emotion as unconditional for more natural speech
        unconditional_keys=["emotion"] if speaker_embedding is None else ["emotion"]
    )

    # Prepare conditioning
    conditioning = model.prepare_conditioning(cond_dict)

    # Generate audio
    print(f"🎵 Generating audio...")
    print(f"📝 Text: '{text[:50]}{'...' if len(text) > 50 else ''}'")
    print(f"🌍 Language: {language}")
    print(f"🎭 Voice: {voice_info}")
    print("⏳ This may take 30-60 seconds depending on text length...")
    
    codes = model.generate(
        prefix_conditioning=conditioning,
        max_new_tokens=min(86 * 30, len(text) * 20),  # Adaptive based on text length
        cfg_scale=2.0,
        batch_size=1,
        progress_bar=True
    )

    # Decode the audio
    print("🔊 Decoding audio...")
    wav_out = model.autoencoder.decode(codes).cpu().detach()
    sr_out = model.autoencoder.sampling_rate
    
    if wav_out.dim() == 2 and wav_out.size(0) > 1:
        wav_out = wav_out[0:1, :]

    # Play the audio
    wav_numpy = wav_out.squeeze().numpy()
    duration = len(wav_numpy) / sr_out
    
    print(f"\n✅ Audio generated successfully!")
    print(f"📊 Sample rate: {sr_out} Hz")
    print(f"⏱️ Duration: {duration:.2f} seconds")
    
    if speaker_embedding is not None:
        print(f"🎭 Successfully used cloned voice!")
    
    # Display audio player
    print(f"\n🎧 Click the play button below to listen:")
    ipd.display(ipd.Audio(wav_numpy, rate=sr_out))
    
    # Store for potential download
    globals()['last_generated_audio'] = (wav_numpy, sr_out, 42)  # seed
    
except Exception as e:
    print(f"❌ Error during audio generation: {e}")
    print("\n🔧 Troubleshooting:")
    print("   • Try shorter text (under 100 characters)")
    print("   • Check GPU memory usage")
    print("   • Make sure the model loaded successfully (Cell 3)")
    print("   • Restart runtime if needed")
    if speaker_embedding is not None:
        print("   • If using cloned voice, try disabling it or re-uploading audio")

print(f"\n💡 Tips:")
print(f"   • Try different text and languages")
print(f"   • Upload different audio files for voice cloning")
print(f"   • Use the 'Advanced Options' cell below for more control")
print(f"   • Save your audio using the 'Save Generated Audio' cell")

# 🎭 Voice Cloning (Optional)

**Upload your own audio to clone any voice!** 

This section allows you to upload a short audio sample (5-30 seconds) of any speaker and use their voice for text-to-speech generation. The voice cloning works with any language and speaking style.

### Requirements for best results:
- **Audio length**: 5-30 seconds (longer is better up to 30 seconds)
- **Audio quality**: Clear speech with minimal background noise
- **Content**: Natural speech (not singing or whispering)
- **Format**: WAV, MP3, or FLAC files
- **Language**: Any language supported by the model

### How it works:
1. Upload your audio file below
2. The model will analyze the speaker's voice characteristics
3. Use the created voice in the text-to-speech cells below

**Privacy Note**: Your uploaded audio is only processed locally in this session and is not stored or shared.

In [ ]:
#@title 4.1 Advanced Text-to-Speech Options (Uses Cloned Voice if Available)

#@markdown ### Adjust model parameters for generation

#@markdown #### Text and language
text = "I can speak with different emotions and characteristics. This is an example of advanced text-to-speech synthesis." #@param {type:"string"}
language = "en-us" #@param ["en-us", "en-gb", "fr-fr", "es-es", "de-de", "it-it", "ja-jp", "zh-cn"]

#@markdown #### Voice Settings  
#@markdown The system will automatically use your cloned voice if available
use_cloned_voice = True #@param {type:"boolean"}

#@markdown #### Emotion controls (0-1 scale)
happiness = 0.7 #@param {type:"slider", min:0, max:1, step:0.05}
sadness = 0.1 #@param {type:"slider", min:0, max:1, step:0.05}
anger = 0.1 #@param {type:"slider", min:0, max:1, step:0.05}
fear = 0.05 #@param {type:"slider", min:0, max:1, step:0.05}
surprise = 0.05 #@param {type:"slider", min:0, max:1, step:0.05}
disgust = 0.05 #@param {type:"slider", min:0, max:1, step:0.05}
other = 0.1 #@param {type:"slider", min:0, max:1, step:0.05}
neutral = 0.3 #@param {type:"slider", min:0, max:1, step:0.05}

#@markdown #### Voice characteristics  
speaking_rate = 15.0 #@param {type:"slider", min:5, max:30, step:0.5}
pitch_std = 45.0 #@param {type:"slider", min:0, max:300, step:5}
fmax = 24000 #@param {type:"slider", min:8000, max:24000, step:1000}
vq_score = 0.78 #@param {type:"slider", min:0.5, max:0.8, step:0.01}
dnsmos_ovrl = 4.0 #@param {type:"slider", min:1, max:5, step:0.1}

#@markdown #### Generation settings
cfg_scale = 2.0 #@param {type:"slider", min:1, max:5, step:0.1}
randomize_seed = True #@param {type:"boolean"}
seed = 42 #@param {type:"integer"}
max_length_multiplier = 20 #@param {type:"slider", min:10, max:50, step:5}

# Check for cloned voice
speaker_embedding = None
voice_info = ""

if use_cloned_voice and 'custom_speaker_embedding' in globals():
    speaker_embedding = custom_speaker_embedding
    if 'speaker_audio_info' in globals():
        info = speaker_audio_info
        voice_info = f"🎭 CLONED VOICE: {info['filename']} ({info['duration']:.1f}s)"
    else:
        voice_info = "🎭 CLONED VOICE from uploaded audio"
    print(voice_info)
else:
    if 'custom_speaker_embedding' in globals():
        print("🔘 Cloned voice available but disabled. Enable 'use_cloned_voice' to use it.")
    else:
        print("🎤 No cloned voice uploaded - using default voice")
    voice_info = "🎤 Default voice"

# Set seed for reproducibility
if not randomize_seed:
    torch.manual_seed(seed)
    used_seed = seed
else:
    used_seed = torch.randint(0, 2**32 - 1, (1,)).item()
    torch.manual_seed(used_seed)

print(f"\n🎲 Using seed: {used_seed}")

# Validate emotion values sum (should be close to 1.0 for best results)
emotion_sum = happiness + sadness + anger + fear + surprise + disgust + other + neutral
if emotion_sum > 1.2 or emotion_sum < 0.8:
    print(f"⚠️ Warning: Emotion values sum to {emotion_sum:.2f}, consider adjusting for better results")

try:
    # Create emotion tensor
    emotion_tensor = torch.tensor([
        float(happiness),  # Happiness
        float(sadness),    # Sadness  
        float(disgust),    # Disgust
        float(fear),       # Fear
        float(surprise),   # Surprise
        float(anger),      # Anger
        float(other),      # Other
        float(neutral)     # Neutral
    ], device=device)

    # Create VQ score tensor (8 values for 8 codebooks)
    vq_tensor = torch.tensor([float(vq_score)] * 8, device=device).unsqueeze(0)

    # Create conditioning dictionary with more parameters
    print(f"\n🔧 Creating advanced conditioning...")
    print(f"📝 Text: '{text[:50]}{'...' if len(text) > 50 else ''}'")
    print(f"🌍 Language: {language}")
    print(f"🎭 Voice: {voice_info}")
    print(f"🎨 Emotions: Happy={happiness:.1f}, Sad={sadness:.1f}, Angry={anger:.1f}, Neutral={neutral:.1f}")
    
    cond_dict = make_cond_dict(
        text=text,
        language=language,
        speaker=speaker_embedding,
        emotion=emotion_tensor,
        speaking_rate=speaking_rate,
        pitch_std=pitch_std,
        fmax=fmax,
        vqscore_8=vq_tensor,
        dnsmos_ovrl=dnsmos_ovrl,
        device=device,
        unconditional_keys=["emotion"] if speaker_embedding is None else []
    )

    # Prepare conditioning
    conditioning = model.prepare_conditioning(cond_dict)

    # Calculate appropriate max_new_tokens based on text length
    estimated_tokens = min(86 * 30, len(text) * max_length_multiplier)
    
    # Generate audio
    print(f"\n🎵 Generating advanced audio...")
    print(f"📊 Text length: {len(text)} chars")
    print(f"📊 Estimated tokens: {estimated_tokens}")
    print(f"📊 CFG scale: {cfg_scale}")
    print("⏳ This may take 30-90 seconds...")
    
    codes = model.generate(
        prefix_conditioning=conditioning,
        max_new_tokens=estimated_tokens,
        cfg_scale=cfg_scale,
        batch_size=1,
        progress_bar=True,
        sampling_params=dict(min_p=0.1, top_k=0, top_p=0.0)  # Use min_p sampling
    )

    # Decode the audio
    print("🔊 Decoding audio...")
    wav_out = model.autoencoder.decode(codes).cpu().detach()
    sr_out = model.autoencoder.sampling_rate
    if wav_out.dim() == 2 and wav_out.size(0) > 1:
        wav_out = wav_out[0:1, :]

    # Play the audio
    wav_numpy = wav_out.squeeze().numpy()
    duration = len(wav_numpy) / sr_out
    
    print(f"\n✅ Advanced audio generated successfully!")
    print(f"📊 Sample rate: {sr_out} Hz")
    print(f"⏱️ Duration: {duration:.2f} seconds")
    print(f"🎭 Voice: {voice_info}")
    print(f"🎛️ Settings: CFG={cfg_scale}, Emotions=[H:{happiness}, S:{sadness}, A:{anger}, N:{neutral}]")
    
    print(f"\n🎧 Click the play button below to listen:")
    ipd.display(ipd.Audio(wav_numpy, rate=sr_out))
    
    # Store for potential download
    globals()['last_generated_audio'] = (wav_numpy, sr_out, used_seed)
    
except Exception as e:
    print(f"❌ Error during advanced audio generation: {e}")
    print("\n🔧 Troubleshooting:")
    print("   • Try simpler emotion settings (closer to default values)")
    print("   • Reduce text length")
    print("   • Lower CFG scale (try 1.5-2.0)")
    print("   • Check GPU memory usage")
    if speaker_embedding is not None:
        print("   • If using cloned voice, try disabling it temporarily")

print(f"\n💡 Advanced Tips:")
print(f"   • Experiment with different emotion combinations")
print(f"   • Adjust speaking rate for faster/slower speech")
print(f"   • Try different CFG scales for varied results")
print(f"   • Use your cloned voice with different emotions!")

In [ ]:
#@title 4.2 Save Generated Audio

#@markdown ### Save and download the generated audio
import os
import scipy.io.wavfile
from datetime import datetime

#@markdown Choose what to save
save_last_generated = True #@param {type:"boolean"}
filename_prefix = "zonos_audio" #@param {type:"string"}
include_timestamp = True #@param {type:"boolean"}
include_settings = True #@param {type:"boolean"}

# Check if we have generated audio from previous cells
try:
    if 'last_generated_audio' in globals():
        wav_numpy, sr_out, used_seed = last_generated_audio
        
        # Create filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S") if include_timestamp else ""
        settings_suffix = f"_seed{used_seed}" if include_settings else ""
        
        if timestamp and settings_suffix:
            filename = f"{filename_prefix}_{timestamp}{settings_suffix}.wav"
        elif timestamp:
            filename = f"{filename_prefix}_{timestamp}.wav"
        elif settings_suffix:
            filename = f"{filename_prefix}{settings_suffix}.wav"
        else:
            filename = f"{filename_prefix}.wav"
        
        # Save the audio file
        print(f"💾 Saving audio as: {filename}")
        scipy.io.wavfile.write(filename, sr_out, wav_numpy)
        
        # Show file info
        file_size = os.path.getsize(filename) / (1024 * 1024)  # MB
        duration = len(wav_numpy) / sr_out
        print(f"✓ Audio saved successfully!")
        print(f"  File: {filename}")
        print(f"  Size: {file_size:.2f} MB")
        print(f"  Duration: {duration:.2f} seconds")
        print(f"  Sample rate: {sr_out} Hz")
        
        # Provide download link
        print("\n📥 Starting download...")
        from google.colab import files
        files.download(filename)
        
        print("🎉 Audio file ready for download!")
        
    elif 'wav_numpy' in globals() and 'sr_out' in globals():
        # Fallback to basic variables if available
        filename = f"{filename_prefix}_basic.wav"
        scipy.io.wavfile.write(filename, sr_out, wav_numpy)
        print(f"✓ Audio saved as {filename}")
        files.download(filename)
        
    else:
        print("❌ No audio has been generated yet.")
        print("Run one of the audio generation cells above first.")
        
except Exception as e:
    print(f"❌ Error saving audio: {e}")
    print("Make sure audio generation completed successfully in previous cells.")

---
**Note:**
- Adjust the import paths and model usage according to your codebase.
- If you encounter issues with dependencies, check the `pyproject.toml` or manually install missing packages.

## Additional Notes and Troubleshooting

### ✅ What This Notebook Does
- Automatically installs all required dependencies including system packages
- Downloads the Zonos transformer model from HuggingFace (2-3 GB)
- Provides both simple and advanced text-to-speech generation
- Supports speaker cloning with uploaded audio files
- Includes a full Gradio web interface for interactive use
- Handles error checking and provides helpful feedback

### 🎯 Performance Tips
- **GPU Runtime**: Use GPU runtime for best performance (Runtime → Change runtime type → Hardware accelerator → GPU)
- **Text Length**: Shorter texts (under 200 characters) generate faster
- **Memory**: The model uses ~2-4 GB of GPU memory when loaded
- **First Run**: Initial model download may take 5-10 minutes depending on connection

### 🔧 Common Issues and Solutions

**Model Loading Fails:**
- Check internet connection stability
- Ensure sufficient GPU/RAM memory (restart runtime if needed)
- Try the transformer model if hybrid fails

**Audio Generation Errors:**
- Reduce text length (try under 100 characters)
- Lower CFG scale (try 1.5 instead of 2.0)
- Simplify emotion settings (use defaults)
- Check GPU memory usage

**Import Errors:**
- Restart runtime and run all cells from the beginning
- Check that installation completed without errors
- Ensure you're using a GPU runtime

**Speaker Cloning Issues:**
- Use audio files 5-30 seconds long
- Ensure audio is clear and contains speech
- Supported formats: WAV, MP3, FLAC
- Try mono audio if stereo doesn't work

### 📚 Model Information
- **Model**: Wamp1re-Ai/Zonos-v0.1-transformer
- **Languages**: English, Japanese, Chinese, French, German
- **Sample Rate**: 44.1 kHz
- **Architecture**: Transformer-based with DAC autoencoder
- **Training Data**: 200k+ hours of multilingual speech

### 🌐 Using Custom Subdomains
When running the Gradio interface, you can optionally use a custom Cloudflare subdomain:
1. Set `use_custom_subdomain = True` in the Gradio cell
2. Choose a unique subdomain name
3. Your interface will be available at `https://your-name.gradio.app`

### 💡 Advanced Usage
For production use or custom applications, consider:
- Using the hybrid model for better quality (requires mamba-ssm)
- Implementing custom conditioning parameters
- Fine-tuning for specific voices or languages
- Using the API programmatically

### 🔗 Useful Links
- [Zonos GitHub Repository](https://github.com/YourUsername/Zonos)
- [Model on HuggingFace](https://huggingface.co/Wamp1re-Ai/Zonos-v0.1-transformer)
- [Zyphra Blog Post](https://www.zyphra.com/post/beta-release-of-zonos-v0-1)
- [Online Playground](https://playground.zyphra.com/audio)

## Using Cloudflare Subdomain for Gradio Interface

When running the Gradio interface, you can use a custom Cloudflare subdomain to make your interface accessible via a consistent URL. This is especially useful for sharing your model with others.

**To use a custom subdomain:**

1. Set the `GRADIO_SUBDOMAIN` environment variable to your desired subdomain name.
2. Set `GRADIO_SHARE=True` to enable sharing.

For example:

In [ ]:
#@title 5. Run Gradio Interface (Interactive Web UI)
import os
import threading
import time

#@markdown ### Gradio Interface Settings
subdomain_name = "my-zonos-app" #@param {type:"string"}
use_custom_subdomain = False #@param {type:"boolean"}
share_publicly = True #@param {type:"boolean"}

# Set environment variables for Gradio
if use_custom_subdomain and subdomain_name:
    os.environ["GRADIO_SUBDOMAIN"] = subdomain_name
    print(f"🌐 Will attempt to use subdomain: {subdomain_name}.gradio.app")
else:
    # Remove subdomain if previously set
    os.environ.pop("GRADIO_SUBDOMAIN", None)

os.environ["GRADIO_SHARE"] = "True" if share_publicly else "False"

print("🚀 Starting Gradio interface...")
print("This may take a moment to initialize...")

# Check if gradio_interface.py exists
if not os.path.exists("gradio_interface.py"):
    print("❌ gradio_interface.py not found!")
    print("Make sure you're in the correct directory and the file exists.")
else:
    try:
        # Import and run the interface
        import subprocess
        import sys
        
        # Run the gradio interface in a separate process
        print("📱 Launching Gradio interface...")
        print("Click on the public URL below to access the web interface")
        print("⚠️ Note: The interface will run until you stop this cell\n")
        
        # Run the gradio interface
        result = subprocess.run([
            sys.executable, "gradio_interface.py"
        ], capture_output=False, text=True)
        
    except KeyboardInterrupt:
        print("\n🛑 Gradio interface stopped by user")
    except Exception as e:
        print(f"❌ Error running Gradio interface: {e}")
        print("\nTroubleshooting:")
        print("- Make sure all dependencies are installed correctly")
        print("- Check that the model loaded successfully in previous cells")
        print("- Try restarting the runtime if needed")

In [ ]:
#@title 🧪 Quick Test - Verify Everything Works
#@markdown Run this cell to verify that Zonos is properly installed and working

print("Running comprehensive test of Zonos installation...")
print("This will check dependencies and try to load the model.")
print()

exec(open('colab_quick_test.py').read())

In [ ]:
#@title 🎭 Voice Cloning - Upload Audio to Clone a Voice

from google.colab import files
import torchaudio
import torch
import IPython.display as ipd
import numpy as np
import os

#@markdown ### Upload Audio File for Voice Cloning
#@markdown Click the "Choose Files" button below to upload an audio file

print("🎤 Voice Cloning Setup")
print("=" * 50)
print("Upload a short audio file (5-30 seconds) of the speaker you want to clone.")
print("")
print("✅ Supported formats: WAV, MP3, FLAC, M4A")
print("✅ Recommended length: 10-30 seconds")
print("✅ Audio should contain clear speech")
print("✅ Minimal background noise preferred")
print("")
print("📁 Click 'Choose Files' to upload your audio:")
print("")

# Upload audio file
uploaded = files.upload()

if uploaded:
    # Process the uploaded file
    speaker_file = list(uploaded.keys())[0]
    file_size = len(uploaded[speaker_file]) / (1024 * 1024)  # MB
    
    print(f"")
    print(f"📁 Processing uploaded file: {speaker_file}")
    print(f"📏 File size: {file_size:.2f} MB")
    print(f"")
    
    try:
        # Load audio file
        print("🔄 Loading audio file...")
        wav, sr = torchaudio.load(speaker_file)
        
        # Get audio info
        duration = wav.shape[1] / sr
        channels = wav.shape[0]
        
        print(f"✅ Audio loaded successfully!")
        print(f"   📊 Duration: {duration:.2f} seconds")
        print(f"   📊 Sample rate: {sr} Hz")
        print(f"   📊 Channels: {channels} ({'Stereo' if channels > 1 else 'Mono'})")
        
        # Check duration
        if duration < 3:
            print(f"⚠️ WARNING: Audio is quite short ({duration:.1f}s). Consider using 5-30 seconds for better results.")
        elif duration > 60:
            print(f"⚠️ WARNING: Audio is long ({duration:.1f}s). Using first 30 seconds for processing.")
            # Trim to first 30 seconds
            max_samples = int(30 * sr)
            wav = wav[:, :max_samples]
            duration = 30.0
            
        # Convert to mono if stereo
        if wav.shape[0] > 1:
            print("🔄 Converting stereo to mono...")
            wav = wav.mean(0, keepdim=True)
        
        # Resample if necessary (model expects certain sample rates)
        target_sr = 44100  # Common target sample rate
        if sr != target_sr:
            print(f"🔄 Resampling from {sr} Hz to {target_sr} Hz...")
            resampler = torchaudio.transforms.Resample(sr, target_sr)
            wav = resampler(wav)
            sr = target_sr
        
        # Play the processed audio for verification
        print(f"")
        print(f"🔊 Preview of processed audio:")
        wav_numpy = wav.squeeze().numpy()
        ipd.display(ipd.Audio(wav_numpy, rate=sr))
        
        # Create speaker embedding
        print(f"")
        print(f"🧠 Creating speaker embedding...")
        print(f"   This analyzes the voice characteristics for cloning")
        
        # Make sure model is loaded
        if 'model' not in globals():
            print(f"❌ Error: Model not loaded yet!")
            print(f"   Please run Cell 3 (Load and run the Zonos model) first.")
        else:
            # Create the speaker embedding
            speaker_embedding = model.make_speaker_embedding(wav, sr)
            speaker_embedding = speaker_embedding.to(device, dtype=torch.bfloat16)
            
            print(f"✅ Speaker embedding created successfully!")
            print(f"   📊 Embedding shape: {speaker_embedding.shape}")
            print(f"   📊 Device: {speaker_embedding.device}")
            
            # Store globally for use in other cells
            globals()['custom_speaker_embedding'] = speaker_embedding
            globals()['speaker_audio_info'] = {
                'filename': speaker_file,
                'duration': duration,
                'sample_rate': sr,
                'channels': channels
            }
            
            print(f"")
            print(f"🎉 Voice cloning setup complete!")
            print(f"")
            print(f"✅ The voice from '{speaker_file}' is now ready for cloning")
            print(f"✅ You can now use this voice in the text-to-speech cells below")
            print(f"✅ The speaker embedding is stored as 'custom_speaker_embedding'")
            print(f"")
            print(f"➡️ Go to the next cell to generate speech with this cloned voice!")
            
    except Exception as e:
        print(f"❌ Error processing audio file: {e}")
        print(f"")
        print(f"💡 Troubleshooting tips:")
        print(f"   • Make sure the file is a valid audio format (WAV, MP3, FLAC)")
        print(f"   • Try a different audio file")
        print(f"   • Ensure the audio contains speech (not music or silence)")
        print(f"   • Check that the model is loaded (run Cell 3 first)")
        
else:
    print(f"")
    print(f"ℹ️ No file uploaded. Voice cloning is optional.")
    print(f"")
    print(f"You can:")
    print(f"✅ Skip this step and use the default voice in the next cells")
    print(f"✅ Come back here later to upload audio for voice cloning")
    print(f"✅ Re-run this cell to upload a different audio file")
    
    # Clear any existing speaker embedding
    if 'custom_speaker_embedding' in globals():
        del globals()['custom_speaker_embedding']
        print(f"🗑️ Cleared previous speaker embedding")

print(f"")
print(f"📝 Next steps:")
print(f"   1. Continue to the next cell for text-to-speech generation")
print(f"   2. The system will automatically use your cloned voice if uploaded")
print(f"   3. You can always come back here to try different voices")